# Feature Extraction

In [1]:
#imports
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk import word_tokenize
import spacy 
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()


In [2]:
df = pd.read_csv('sentence_classifications_.csv')

In [3]:
df.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,prop_start_char,prop_end_char,prop_txt_snippet,sent_#
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,NaN,NaN,1
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",111.0,143.0,Grand Wizard of the Ku Klux Klan,2
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",305.0,313.0,enamored,3
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,NaN,NaN,4
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,NaN,NaN,5


In [4]:
df_1 = df[df['text'].isna()==False]
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15220 entries, 0 to 15240
Data columns (total 11 columns):
Unnamed: 0             15220 non-null int64
article_id             15220 non-null int64
sentence_char_start    15220 non-null int64
sentence_char_end      15220 non-null int64
propaganda             15220 non-null object
propaganda_type        3840 non-null object
text                   15220 non-null object
prop_start_char        3840 non-null float64
prop_end_char          3840 non-null float64
prop_txt_snippet       3840 non-null object
sent_#                 15220 non-null int64
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


In [5]:
#LOOKING FOR DIFFERENCES IN PROP AND NON PROP FOR CERTAIN WORDS/TOPICS... MIGHT NEED TO CHANGE SOME LABELS

In [6]:
non_prop = df_1[(df_1['propaganda']=='non-propaganda')]
prop = df_1[(df_1['propaganda']=='propaganda')]


## Feature Extraction From Sentences

### Sentence Sentiment Score

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [8]:
def sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt

In [9]:
def sentence_sentiment_score(new_col, transform_col, df):
    df[new_col] = df[transform_col].apply(lambda x: sentiment_scores(x)['compound'])
    return df

In [10]:
#create new column with vader sentiment score for each sentence
df_1 = sentence_sentiment_score('sentiment_score','text',df_1)

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
#create new column with absolute value of vader sentiment score for each sentence
df_1['abs_sent_score'] = df_1['sentiment_score'].apply(lambda x: abs(x))

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
df_1.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,prop_start_char,prop_end_char,prop_txt_snippet,sent_#,sentiment_score,abs_sent_score
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,NaN,NaN,1,0.0000,0.0000
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",111.0,143.0,Grand Wizard of the Ku Klux Klan,2,0.5423,0.5423
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",305.0,313.0,enamored,3,0.3612,0.3612
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,NaN,NaN,4,0.0000,0.0000
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,NaN,NaN,5,0.0000,0.0000


### Sentence Punctuation Count

In [13]:
import string  
punctuation = string.punctuation

In [14]:
punctuation = punctuation + '”'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~”'

In [15]:
def count_punct(text):
    count = 0
    for char in text:
        if char in punctuation:
            count +=1
    return count

def count_punct_col(df,transform_col,new_col):
    df[new_col]=df[transform_col].apply(lambda x: count_punct(x))
    return df
    

In [16]:
test_string = 'He said: "HELLO!"... but that is it... no more'
count_punct(test_string)

10

In [17]:
df_1 = count_punct_col(df_1,'text','punct_count')

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
df_1.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,prop_start_char,prop_end_char,prop_txt_snippet,sent_#,sentiment_score,abs_sent_score,punct_count
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,NaN,NaN,1,0.0000,0.0000,0
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",111.0,143.0,Grand Wizard of the Ku Klux Klan,2,0.5423,0.5423,4
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",305.0,313.0,enamored,3,0.3612,0.3612,4
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,NaN,NaN,4,0.0000,0.0000,5
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,NaN,NaN,5,0.0000,0.0000,1


### Simple Tokenizer for Word Count, Word Length, & POS counter

In [19]:
# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

In [20]:
contr_dict={"I’m": "I am",
            "won’t": "will not",
            "’s" : "", 
            "’ll":"will",
            "’ve ":"have ",
            "n’t":"not",
            "’re": "are",
            "’d": "would",
            "y’all": "all of you",
            "I'm": "I am",
            "won't": "will not",
            "'s" : "", 
            "'ll":"will",
            "'ve ":"have ",
            "n't":"not",
            "'re": "are",
            "'d": "would",
            "y'all": "all of you"}
contr_dict.keys()


dict_keys(['I’m', 'won’t', '’s', '’ll', '’ve ', 'n’t', '’re', '’d', 'y’all', "I'm", "won't", "'s", "'ll", "'ve ", "n't", "'re", "'d", "y'all"])

In [21]:
def replace_contractions(sentence, contr_dict=contr_dict):
    for contr in contr_dict.keys():
        if contr in sentence:
            sentence = sentence.replace(contr,contr_dict[contr])
    return sentence

In [22]:
sent_1 = '''After a period of review, the professor changed Arnold's grade to a B+, according to BuzzFeed News.'''

In [23]:
replace_contractions(sent_1,contr_dict)

'After a period of review, the professor changed Arnold grade to a B+, according to BuzzFeed News.'

In [24]:
def simple_tokenizer(text):
    text = replace_contractions(text,contr_dict)
    mytokens = nlp(text)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [word for word in mytokens if word not in punctuation]
    return mytokens

In [25]:
test_tokens = simple_tokenizer('''South Florida Muslim Leader Sofian Zakkout’s David Duke Day''')

In [26]:
test_tokens

['south',
 'florida',
 'muslim',
 'leader',
 'sofian',
 'zakkout',
 'david',
 'duke',
 'day']

### Sentence Word Count

In [27]:
def word_count(tokens):
    return len(tokens)

In [28]:
word_count(test_tokens)

9

In [29]:
def word_count_col(df, new_col, transform_col):
    df[new_col] = df[transform_col].apply(lambda x: word_count(simple_tokenizer(x)))
    return df


In [30]:
df_1 = word_count_col(df_1,'word_count','text')

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Part of Speech Counter

### POS: Proportion of Adj, Nouns, Verbs, Adverbs

In [32]:
def adj_counter(text):
    text = replace_contractions(text,contr_dict)
    mytokens = nlp(text)
    adjs = [word for word in mytokens if word.pos_ == "ADJ"]
    return (len(adjs))

In [33]:
def verb_counter(text):
    text = replace_contractions(text,contr_dict)
    mytokens = nlp(text)
    verbs = [word for word in mytokens if word.pos_ == "VERB"]
    return (len(verbs))

In [34]:
def adv_counter(text):
    text = replace_contractions(text,contr_dict)
    mytokens = nlp(text)
    advs = [word for word in mytokens if word.pos_ == "ADV"]
    return (len(advs))

In [35]:
def noun_counter(text):
    text = replace_contractions(text,contr_dict)
    mytokens = nlp(text)
    nouns = [word for word in mytokens if word.pos_ == "NOUN"]
    return (len(nouns))

In [36]:
a = adj_counter('She is a very tall person')
v = verb_counter('She is a very tall person')
d = adv_counter('She is a very tall person')
n = noun_counter('She is a very tall person')
print(a,v,d,n)

1 0 1 1


In [37]:
df_1['%adj'] = df_1.text.apply(lambda x: adj_counter(x)/word_count(x))
df_1['%verb'] = df_1.text.apply(lambda x: verb_counter(x)/word_count(x))
df_1['%adv'] = df_1.text.apply(lambda x: adv_counter(x)/word_count(x))
df_1['%noun'] = df_1.text.apply(lambda x: noun_counter(x)/word_count(x))

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

### Sentence Avg. Word Length

In [38]:
def avg_word_length(tokens):
    word_lengths = [len(token) for token in tokens]
    return np.mean(word_lengths)

In [39]:
avg_word_length(test_tokens)

5.444444444444445

In [40]:
def avg_word_length_col(df, new_col, transform_col):
    df[new_col] = df[transform_col].apply(lambda x: avg_word_length(simple_tokenizer(x)))
    return df

In [41]:
df_1 = avg_word_length_col(df_1, 'avg_word_length', 'text')

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Number of Strong Subjectives in Sentence

In [110]:
import spacy
nlp = spacy.load("en_core_web_sm")
from nltk import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

punctuation = string.punctuation
punctuation = punctuation+"..."+"--"+"“"+"”"+"``"+"''"+"’"+"–"+"—"+"‘"
lemmatizer = WordNetLemmatizer()

In [111]:
# function to convert nltk tag to wordnet tag
# this is important because having the POS tag improves lemmatization
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [112]:
#tokenization and lemmatization function
def tokenize_sentence(sentence):
    #remove contractions
    sentence = replace_contractions(sentence, contr_dict=contr_dict)
    
    #tokenize the sentence
    mytokens = nltk.word_tokenize(sentence)
    
    #tag tokens with part of speech
    nltk_tagged = nltk.pos_tag(mytokens)
    
    # remove punctuation
    nltk_tagged = [ word for word in nltk_tagged if word[0] not in punctuation ]
    
    # strip all tokens and make lowercase 
    nltk_tagged = [ (word[0].lower().strip(),word[1]) for word in nltk_tagged ]
    
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    
    lemmatized_tokens = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_tokens.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_tokens.append(lemmatizer.lemmatize(word, tag))
            
        lemmatized_tokens = [word for word in lemmatized_tokens if word not in stop_words]
    return lemmatized_tokens


In [121]:
strong_subjectives =  open('''/Users/sashaepelbaum/Desktop/DATA_SCIENCE/PROJECTS/nlp/neutralizing-bias/src/lexicons/strong_subjectives_riloff2003.txt''', "r")

In [122]:
strong_subjectives =strong_subjectives.read()

In [123]:
strong_subjective_lemmas = tokenize_sentence(strong_subjectives.replace('\n', ' '))

In [124]:
strong_subjectives_lemmas

['irreconcilable',
 'accede',
 'overjoy',
 'madden',
 'torturously',
 'exuberant',
 'soften',
 'inflict',
 'rewarding',
 'anarchism',
 'indicative',
 'obviate',
 'quandary',
 'culpable',
 'allegation',
 'feel',
 'polluter',
 'satisfactory',
 'geezer',
 'worthlessness',
 'lack',
 'embroil',
 'complain',
 'disagreeable',
 'grace',
 'madly',
 'serious',
 'distress',
 'sagacity',
 'contentious',
 'luck',
 'adorer',
 'declaim',
 'fame',
 'quite',
 'tidy',
 'view',
 'disorient',
 'dungeon',
 'utterance',
 'inequality',
 'yep',
 'appal',
 'exterminate',
 'attentive',
 'indescribably',
 'mindless',
 'imploringly',
 'magnificent',
 'strife',
 'astound',
 'dishonesty',
 'impatiently',
 'nag',
 'draconic',
 'adulatory',
 'joyous',
 'prate',
 'spookily',
 'underdog',
 'excessive',
 'baffle',
 'oddity',
 'untrue',
 'damn',
 'gabble',
 'sensibly',
 'gravely',
 'puppet',
 'scorch',
 'eulogize',
 'drone',
 'mellow',
 'blister',
 'cataclysmically',
 'harsh',
 'assiduous',
 'fantastic',
 'menacingly',
 

In [102]:
# strong_subjectives_lemmas = [ word.lemma_.lower() for word in strong_subjectives ]

In [104]:
# strong_subjectives_lemmas = [word for word in strong_subjectives_lemmas if word != '-']

In [135]:
def count_strong_subjectives(text):
    counter = 0
    for token in text:
        if token in strong_subjective_lemmas:
            counter += 1
    return counter



In [140]:
df_1['strong_subjectives_count'] = df_1['text'].apply(lambda text: count_strong_subjectives(tokenize_sentence(text)))

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
df_1.strong_subjectives_count.unique()

array([ 0,  2,  3,  1,  4,  5, 10,  7,  9,  6,  8, 13, 11, 12])

### Sample Row for Propaganda

In [142]:
df_1[df_1['propaganda']=='propaganda'].loc[1]

Unnamed: 0                                                                  1
article_id                                                          701225819
sentence_char_start                                                        60
sentence_char_end                                                         207
propaganda                                                         propaganda
propaganda_type                                         Name_Calling,Labeling
text                        David Duke, the white supremacist icon and for...
prop_start_char                                                           111
prop_end_char                                                             143
prop_txt_snippet                             Grand Wizard of the Ku Klux Klan
sent_#                                                                      2
sentiment_score                                                        0.5423
abs_sent_score                                                  

### Sample Row for Non-Propaganda

In [143]:
df_1[df_1['propaganda']=='non-propaganda'].loc[3]

Unnamed: 0                                                                  3
article_id                                                          701225819
sentence_char_start                                                       382
sentence_char_end                                                         525
propaganda                                                     non-propaganda
propaganda_type                                                           NaN
text                        Last month, once again, Zakkout chose to showc...
prop_start_char                                                           NaN
prop_end_char                                                             NaN
prop_txt_snippet                                                          NaN
sent_#                                                                      4
sentiment_score                                                             0
abs_sent_score                                                  

### Looking for Columns with Missing Values

In [144]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15220 entries, 0 to 15240
Data columns (total 21 columns):
Unnamed: 0                  15220 non-null int64
article_id                  15220 non-null int64
sentence_char_start         15220 non-null int64
sentence_char_end           15220 non-null int64
propaganda                  15220 non-null object
propaganda_type             3840 non-null object
text                        15220 non-null object
prop_start_char             3840 non-null float64
prop_end_char               3840 non-null float64
prop_txt_snippet            3840 non-null object
sent_#                      15220 non-null int64
sentiment_score             15220 non-null float64
abs_sent_score              15220 non-null float64
punct_count                 15220 non-null int64
word_count                  15220 non-null int64
%adj                        15220 non-null float64
%verb                       15220 non-null float64
%adv                        15220 non-null flo

### Dropping Non-Informative Columns

In [145]:
df_2 = df_1.drop(['Unnamed: 0','sentence_char_start','sentence_char_end',
                  'prop_start_char','prop_end_char'], axis=1)

In [146]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15220 entries, 0 to 15240
Data columns (total 16 columns):
article_id                  15220 non-null int64
propaganda                  15220 non-null object
propaganda_type             3840 non-null object
text                        15220 non-null object
prop_txt_snippet            3840 non-null object
sent_#                      15220 non-null int64
sentiment_score             15220 non-null float64
abs_sent_score              15220 non-null float64
punct_count                 15220 non-null int64
word_count                  15220 non-null int64
%adj                        15220 non-null float64
%verb                       15220 non-null float64
%adv                        15220 non-null float64
%noun                       15220 non-null float64
avg_word_length             15172 non-null float64
strong_subjectives_count    15220 non-null int64
dtypes: float64(7), int64(5), object(4)
memory usage: 2.0+ MB


### Dropping ~50 Rows With Only Punct in Text

In [148]:
df_2 = df_2[df_2['avg_word_length'].isna() == False]

In [149]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15172 entries, 0 to 15240
Data columns (total 16 columns):
article_id                  15172 non-null int64
propaganda                  15172 non-null object
propaganda_type             3840 non-null object
text                        15172 non-null object
prop_txt_snippet            3840 non-null object
sent_#                      15172 non-null int64
sentiment_score             15172 non-null float64
abs_sent_score              15172 non-null float64
punct_count                 15172 non-null int64
word_count                  15172 non-null int64
%adj                        15172 non-null float64
%verb                       15172 non-null float64
%adv                        15172 non-null float64
%noun                       15172 non-null float64
avg_word_length             15172 non-null float64
strong_subjectives_count    15172 non-null int64
dtypes: float64(7), int64(5), object(4)
memory usage: 2.0+ MB


In [150]:
df_2.to_csv('meta_features.csv',index=False)